In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#reading the required dataset
data=pd.read_csv('/kaggle/input/clean-kenyan-dataset/Clean_Kenya_Tourism_datasets (2).csv')

In [ ]:
#checking the first five rows of the training data
data.head()

In [ ]:
#checking the columns in the training dataset
data.columns

In [ ]:
#checking the information of the training dataset
data.info()

In [ ]:
#step 2 is to clean the data
#replacing the nan values in the travel_with column of the training data with Alone

data['travel_with'] = data['travel_with'].replace(np.nan, 'Alone')

In [ ]:
#replacing the nan values in the total_female column of the training data with 1.0

data['total_female'] = data['total_female'].replace(np.nan, 1.0)

In [ ]:
#replacing the nan values in the total_male column of the training data with 1.0

data['total_male'] = data['total_male'].replace(np.nan, 1.0)

In [ ]:
#replacing the nan values in the most_impressing column of the training data with No comments

data['most_impressing'] = data['most_impressing'].replace(np.nan, 'No comments')

In [ ]:
#replacing 24-jan to 1-24 to correlate with 

data['age_group'] = data['age_group'].replace('24-Jan', '1-24')

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

In [ ]:
data.head()

In [ ]:
# %convert float dtypes to int[total_female,total_male,night_mainland,night_zanzibar]
data["total_female"] = data['total_female'].astype('int')
data["total_male"] = data['total_male'].astype('int')
data["night_mainland"] = data['night_mainland'].astype('int')


In [ ]:
# %Let's generate new features from some columns which makes some sense
data["total_people"] = data["total_female"] + data["total_male"]

data["total_nights"] = data["night_mainland"] 

In [ ]:
# %more information about data
data.info()

In [ ]:
# %Before hand let's remove ID Column
data.drop('ID', axis='columns', inplace=True)

In [ ]:
data.head()

In [ ]:
# %then it's time to encode objects into numeric

for colname in data.select_dtypes("object"):
    data[colname],_=data[colname].factorize()

In [ ]:
data.head()

In [ ]:
# Step 3 model building
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# %spliting dependent and independent features
# Drop the "total_cost" column
features_cols = data.drop(columns=["total_cost"])

# Now 'features_cols' contains the DataFrame without the "total_cost" column

cols = features_cols.columns
target=data["total_cost"]

In [ ]:
data[cols].shape , target.shape

((4809, 22), (4809,))

In [ ]:
#modelling
# %Let's set seed for reproducibility of our final model
SEED = 2020 #you can set any number
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(data[cols],target, test_size=0.20, random_state = SEED)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(3847, 23) (3847,)
(962, 23) (962,)

In [ ]:
#LINEAR REGRESSION
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error 
# %Model initialization & training
LR = LinearRegression().fit(X_train,y_train)

# %Predictions
y_train_pred = LR.predict(X_train)
y_test_pred = LR.predict(X_test)

# % Evaluation
mae = mean_absolute_error(y_test, y_test_pred)
print('Using scikit-lean, the mae error is {}'.format(mae))

Using scikit-lean, the mae error is 5742313.756659567

In [ ]:
# % Evaluation
mse = mean_squared_error(y_test, y_test_pred)
print('Using scikit-lean, the mse error is {}'.format(mse))

In [ ]:
#EXTREME GRADIENT BOOST
from xgboost import XGBRegressor


In [ ]:
# %instatiate the model
XGB = XGBRegressor()

# %training the model
XGB.fit(X_train, y_train)

# %prediction
y_train_pred = XGB.predict(X_train)
y_test_pred = XGB.predict(X_test)

# % Evaluation
mae = mean_absolute_error(y_test, y_test_pred)
print('Using scikit-lean, the mae error is {}'.format(mae))

mse = mean_squared_error(y_test, y_test_pred)
print('Using scikit-lean, the mse error is {}'.format(mse))

Here we can observe changes XGBRegression shows to perform well compared to Linear Regression even without passing some parameters.

Not bad for such a lazy implementation, even without data normalization After all, the data will not always be so "good". So don't forget to pre-process the data. Now let's apply polynomial features technique on the data. And look at the result.

APPLYING POLYNOMIAL FEATURES
Polynomial features are those features created by raising existing features to an exponent.

For example, if a dataset had one input feature X, then a polynomial feature would be the addition of a new feature (column) where values were calculated by squaring the values in X, e.g. X^2. This process can be repeated for each input variable in the dataset, creating a transformed version of each.

As such, polynomial features are a type of feature engineering, e.g. the creation of new input features based on the existing features.

The “degree” of the polynomial is used to control the number of features added, e.g. a degree of 3 will add two new variables for each input variable. Typically a small degree is used such as 2 or 3.

*Generally speaking, it is unusual to use d greater than 3 or 4 because for large values of d, the polynomial curve can become overly flexible and can take on some very strange shapes.*

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# % applying polynomial features with 2 degree
quad = PolynomialFeatures (degree = 2)
x_quad = quad.fit_transform(data[cols])

X_train,X_test,y_train,y_test = train_test_split(x_quad,target, random_state = 0)
# %instatiate the model
XGB_par_ = XGBRegressor(n_estimators= 100, colsample_bynode = 0.8, learning_rate = 0.02,max_depth =  7)

# %training the model
XGB_par_.fit(X_train, y_train)

# %prediction
y_train_pred = XGB_par_.predict(X_train)
y_test_pred = XGB_par_.predict(X_test)

# % Evaluation
mae = mean_absolute_error(y_test, y_test_pred)
print('Using scikit-lean, the mae error is {}'.format(mae))

mse = mean_squared_error(y_test, y_test_pred)
print('Using scikit-lean, the mse error is {}'.format(mse))

Using scikit-lean, the mae error is 5370262.699263508
Using scikit-lean, the mse error is 98250845059727.06

EXTREME GRADIENT BOOST WITH PARAMETERS

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[cols],target, test_size=0.20, random_state = SEED)

# %instatiate the model
XGB_par = XGBRegressor( n_estimators= 100, colsample_bynode = 0.8, learning_rate = 0.02,max_depth =  7)

# %training the model
XGB_par.fit(X_train, y_train)

# %prediction
y_train_pred = XGB_par.predict(X_train)
y_test_pred = XGB_par.predict(X_test)

# % Evaluation
mae = mean_absolute_error(y_test, y_test_pred)
print('Using scikit-lean, the mae error is {}'.format(mae))

mse = mean_squared_error(y_test, y_test_pred)
print('Using scikit-lean, the mse error is {}'.format(mse))

Using scikit-lean, the mae error is 5109772.414642411
Using scikit-lean, the mse error is 93266365624137.86

Now we can see polynomial features technique does not improve the performance real well compared to Extreme gradient boost with parameter only. This shows that most of features were categorical applying polynomial feature engineering not the best approach ,may be you can try applying only to those numeric features and then concatinate with others

VISUALIZE PREDICTIONS

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,7))

plt.scatter(y_train_pred,y_train_pred - y_train,
          c = 'black', marker = 'o', s = 50, alpha = 0.5,
          label = 'Train data')
plt.scatter(y_test_pred,y_test_pred - y_test,
          c = 'g', marker = 'o', s = 50, alpha = 0.7,
          label = 'Test data')
plt.xlabel('Predicted values')
plt.ylabel('Tailings')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
#Let's Visualize on the best contributed features
feat_importances = pd.Series(XGB_par.feature_importances_, index=cols)
plt.figure(figsize=(15,10))
plt.title("FEATURE IMPORTANCE PLOT")
feat_importances.nlargest(18).plot(kind='barh')
plt.show()

In [51]:
#To create a Streamlit app for predicting the cost for a tourist, you can follow these steps. Note that you'll need to have Streamlit installed in your Python environment. You can install it using `pip install streamlit`.

#1. Create a new Python file, e.g., `tourist_cost_app.py`.
#2. Copy and paste the following code into your file:

#```python
# Importing the required libraries
import streamlit as st
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/clean-kenyan-dataset/Clean_Kenya_Tourism_datasets (2).csv')

# Cleaning the data
# ... (Copy the data cleaning code from your original code)

# Convert categorical features to numeric
for colname in data.select_dtypes("object"):
    data[colname], _ = data[colname].factorize()

# Feature engineering
data["total_people"] = data["total_female"] + data["total_male"]
data["total_nights"] = data["night_mainland"]
data.drop('ID', axis='columns', inplace=True)

# Split the data
features_cols = data.drop(columns=["total_cost"])
cols = features_cols.columns
target = data["total_cost"]
X_train, X_test, y_train, y_test = train_test_split(data[cols], target, test_size=0.20, random_state=2020)

# Model training
XGB_par = XGBRegressor(n_estimators=100, colsample_bynode=0.8, learning_rate=0.02, max_depth=7)
XGB_par.fit(X_train, y_train)

# Streamlit app
st.title("Tourist Cost Prediction App")

# Sidebar with input parameters
st.sidebar.header("Input Parameters")
country = st.sidebar.selectbox("Country", data["country"].unique())
age_group = st.sidebar.selectbox("Age Group", data["age_group"].unique())
travel_with = st.sidebar.selectbox("Travel With", data["travel_with"].unique())
total_female = st.sidebar.slider("Total Female", min_value=0, max_value=10, step=1, value=1)
total_male = st.sidebar.slider("Total Male", min_value=0, max_value=10, step=1, value=1)
purpose = st.sidebar.selectbox("Purpose", data["purpose"].unique())
main_activity = st.sidebar.selectbox("Main Activity", data["main_activity"].unique())
info_source = st.sidebar.selectbox("Info Source", data["info_source"].unique())
tour_arrangement = st.sidebar.selectbox("Tour Arrangement", data["tour_arrangement"].unique())
package_transport_int = st.sidebar.selectbox("Package Transport", data["package_transport_int"].unique())
night_mainland = st.sidebar.slider("Night Mainland", min_value=0, max_value=30, step=1, value=10)
payment_mode = st.sidebar.selectbox("Payment Mode", data["payment_mode"].unique())
first_trip_Kenya = st.sidebar.selectbox("First Trip to Kenya", data["first_trip_Kenya"].unique())
most_impressing = st.sidebar.selectbox("Most Impressing", data["most_impressing"].unique())

# User input dictionary
user_input = {
    "country": country,
    "age_group": age_group,
    "travel_with": travel_with,
    "total_female": total_female,
    "total_male": total_male,
    "purpose": purpose,
    "main_activity": main_activity,
    "info_source": info_source,
    "tour_arrangement": tour_arrangement,
    "package_transport_int": package_transport_int,
    "night_mainland": night_mainland,
    "payment_mode": payment_mode,
    "first_trip_Kenya": first_trip_Kenya,
    "most_impressing": most_impressing,
}

# Create a DataFrame from the user input
user_input_df = pd.DataFrame([user_input])

# Convert categorical features to numeric
for colname in user_input_df.select_dtypes("object"):
    user_input_df[colname], _ = user_input_df[colname].factorize()

# Feature engineering
user_input_df["total_people"] = user_input_df["total_female"] + user_input_df["total_male"]
user_input_df["total_nights"] = user_input_df["night_mainland"]
user_input_df.drop(columns=["total_female", "total_male", "night_mainland"], inplace=True)

# Predict the cost
prediction = XGB_par.predict(user_input_df)[0]

# Display the prediction
st.header("Predicted Total Cost:")
st.write(f"${prediction:,.2f}")

# Visualize predictions
st.sidebar.header("Model Evaluation")
show_plot = st.sidebar.checkbox("Show Predictions vs Residuals")

ModuleNotFoundError: No module named 'streamlit'